# Postgres Methods & Experimentation

In [1]:
import sys
import json
# import redis

# from qdrant_client import QdrantClient
from pathlib import Path
from typing import Optional, Sequence

# Add parent directory to path to import from implementation package
# Notebooks are in implementation/notebooks/, so we go up two levels to project root
sys.path.insert(0, str(Path().resolve().parent.parent))

from implementation.classes.movie import BaseMovie
from db.ingest_movie import ingest_movie
from db.postgres import pool

# # Redis
# r = redis.Redis(host="localhost", port=6379, decode_responses=True)

# # Qdrant
# qdrant = QdrantClient(host="localhost", port=6333)

In [2]:
# Open the pool and establish initial connections
await pool.open()
# Validate that connections actually work (fast-fail if Postgres is unreachable)
await pool.check()

In [3]:
# LOAD MOVIES

json_path = Path("../../saved_imdb_movies.json")
with open(json_path, "r", encoding="utf-8") as f:
    movies_data = json.load(f)

# Convert each dictionary to an IMDBMovie object
movies = [BaseMovie(**movie_dict) for movie_dict in movies_data]

In [4]:
movie_to_ingest = movies[0]

print(f"Ingesting movie: {movie_to_ingest.title}")

await ingest_movie(movie_to_ingest)

Ingesting movie: ferris bueller's day off


In [ ]:
# # Gracefully close all connections on shutdown
# await pool.close()